# 전환이 일어나지 않은 유저(only view)에게 view_count를 사용하여 추천

# 라이브러리 로드

In [1]:
!pip install implicit
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit
import datetime
from tqdm import tqdm

# 데이터 로드

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')


# 11월 넘어가는 유저 제거

In [4]:
df['event_time'] = df['event_time'].apply(lambda x : x[:-4]).astype('datetime64')
df['event_time'].dtype

dtype('<M8[ns]')

In [5]:
df['event_time'] = df['event_time'] + datetime.timedelta(hours=4)
df = df.loc[df['event_time'] < '2019-11-01']

In [6]:
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
42398925,2019-10-31 23:59:59,view,12718908,452985123,NaN,yokohama,96.269997,536689953,0a77064e-0bc6-4745-b703-5e8208bf8f6e
42398926,2019-10-31 23:59:59,view,28719204,-352320551,apparel.shoes,alpina,220.080002,534245704,699164ab-a8f6-4ce0-bd07-b059ff3e1db2
42398927,2019-10-31 23:59:59,view,1005217,-1769995873,electronics.smartphone,xiaomi,151.070007,560095437,15e037ed-4f14-44ba-b7c0-0c033a6c3649
42398928,2019-10-31 23:59:59,view,1003892,-1769995873,electronics.smartphone,xiaomi,144.729996,566243306,566c6d0a-2cd5-4248-ac7b-3af675c05fd3
42398929,2019-10-31 23:59:59,view,17100010,947913283,NaN,NaN,77.199997,512824822,dee78bdc-718a-42bd-8b78-a26259cd1078


# 데이터 전처리

In [7]:
data = df[['user_id','product_id','event_type']]

## view만 존재하는 유저 데이터 추출

1. cart, purchase 이력이 있는 유저들의 id값 받아오기
2. 위에서 얻은 cart, purchase 이력이 있는 유저들을 제외하기

In [8]:
drop_user_id = data.loc[data['event_type'] != 'view', 'user_id']

In [9]:
data = data.loc[~data['user_id'].isin(drop_user_id)].reset_index(drop=True)
data['event_type'].unique()

['view']
Categories (3, object): ['cart', 'purchase', 'view']

In [10]:
# event_type dtype 을 object로 변경

data['event_type'] = data['event_type'].astype('object')

## view 수 구하기

In [11]:
grouped = data.groupby(['user_id','product_id'])['event_type'].count()
grouped =grouped.reset_index()
data = grouped.rename(columns = {'event_type' : 'view_count'})
data.head()

,user_id,product_id,view_count
0,33869381,7002639,1
1,64078358,10600284,1
2,183503497,22200103,1
3,184265397,6902133,2
4,184265397,6902303,2


# Item Lookup(prorduct_lookup) 테이블 만들기

In [12]:
product_lookup = df[['product_id','category_code','brand']].drop_duplicates('product_id').reset_index(drop=True).sort_values('product_id')
product_lookup.head()

,product_id,category_code,brand
151915,1000978,electronics.smartphone,NaN
8437,1001588,electronics.smartphone,meizu
85152,1001606,electronics.smartphone,apple
32556,1002042,electronics.smartphone,samsung
9400,1002062,electronics.smartphone,samsung


# Rating Matrix 만들기

In [13]:
num_user = data['user_id'].nunique()
num_item = data['product_id'].nunique()
num_user, num_item

(2538082, 159250)

In [14]:
users = list(np.sort(data['user_id'].unique()))
products = list(data['product_id'].unique())
count = list(data['view_count'])

rows = data['user_id'].astype('category').cat.codes
data['user_id_code'] = data['user_id'].astype('category').cat.codes

cols = data['product_id'].astype('category').cat.codes
data['product_id_code'] = data['product_id'].astype('category').cat.codes

len(users), len(products), len(count)

(2538082, 159250, 15346917)

In [15]:
user_item_matrix = sparse.csr_matrix((count, (rows, cols)), shape=(num_user, num_item))
user_item_matrix

<2538082x159250 sparse matrix of type '<class 'numpy.int64'>'
	with 15346917 stored elements in Compressed Sparse Row format>

# 추천 시스템 class 구현

In [16]:
from implicit.als import AlternatingLeastSquares
from sklearn import metrics
class MyALS():
  """
  implicit 라이브러리를 이용하여 필요한 기능을 담았습니다.

  Parameters
  ----------
  model : implicit 라이브러리의 AlternatingLeastSquares() 클래스로 생성된 인스턴스
  user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
  item_lookup : item(product)에 대한 정보를 담은 테이블
  data : User-Item Matrix를 만든 원본데이터

  """
  def __init__(self, model, user_item_matrix, item_lookup, data):

    self.model = model
    self.metrics_model = model
    self.user_item_matrix = user_item_matrix
    self.item_lookup = item_lookup
    self.data = data

  def fit(self, user_item_matrix):
    """
    행렬분해의 ALS를 이용하여 모델을 학습합니다.

    Parameters
    ----------
    user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
    """
    self.model.fit(user_item_matrix)
    
  def _user_id_2_code(self, user_id):
    """
    입력받은 user_id를 User_Item_Matrix에 있는 user_id_code로 바꾸어주는 함수

    Parameters
    ----------
    user_id : 유저 ID

    """
    user_id_code = self.data.loc[self.data['user_id'] == user_id, 'user_id_code'].unique()[0]
    return user_id_code

  def _product_id_2_code(self, product_id):
    """
    입력받은 product_id를 User-Item-Matrix에 있는 product_id_code로 바꾸어주는 함수

    Parameters
    ----------
    product_id : 상품 ID
    """
    product_id_code = self.data.loc[self.data['product_id'] == product_id, 'product_id_code'].unique()[0]
    return product_id_code

  def _code_2_product_id(self, product_id_code):
    """
    입력받은 product_id_code를 User-Item-Matrix에 있는 product_id로 바꾸어주는 함수

    Parameters
    ----------
    product_id_code : 상품 ID code    
    """
    product_id = self.data.loc[self.data['product_id_code'] == product_id_code, 'product_id'].unique()[0]
    return product_id

  def get_recom_product(self, user_id, n = 10):
    """  
    user_id에 맞는 product를 n개 만큼 추천하여 데이터프레임 형태로 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    n : 추천 받게 될 item의 수
    """ 

    # user_id_2_code 함수를 이용하여 유저의 ID를 user_id_code로 변환합니다
    user_id_code = self._user_id_2_code(user_id)
  
    # model의 recommend를 이용하여 추천받는 제품의 id를 추출합니다.
    # 이때 추천 받는 제품의 id는 product_id가 아니라 product_id_code 입니다.
    recommended = self.model.recommend(user_id_code, self.user_item_matrix[user_id_code], N=n)[0]
    #결과를 담을 리스트를 초기화 합니다.
    results = []
    # 추천 받은 id를 돌면서 item_lookup 테이블에서 해당 product의 정보를 찾아 결과에 담습니다.
    for product_id_code in recommended:
      
      recommended_product_id = self._code_2_product_id(product_id_code)
      result = self.item_lookup.loc[self.item_lookup['product_id'] == recommended_product_id]
      results.append(result)
      
    return pd.concat(results)
  
  def get_user_topN_product(self,user_id,column, n = 20):
    """
    유저가 특정 기준값이 높은 제품 N개를 반환

    Parameters
    ----------
    user_id : 유저 ID
    column : 값을 확인할 기준이 되는 컬럼
    n : 반환할 Item 수
    """
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 product_id를 추출
    product_ids = self.data.loc[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n]['product_id'].values
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 column을 추출
    product_values = self.data[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n][column].values

    results = []
    #item_lookup 테이블에서 id에 맞는 데이터프레임을 찾음
    for i in product_ids:
      result = self.item_lookup.loc[self.item_lookup['product_id'] == i]
      results.append(result)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame[column] = product_values

    return frame

  def get_explain(self, user_id, item_id, column):
    """
    사용자에게 제품이 추천된 이유를 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    item_id : Item(product) ID
    column : 확인할 컬럼
    """
    #입력받은 user_id, item_id를 user_id_code, product_id_code로 바꾸어줌
    user_id_code = self._user_id_2_code(user_id)
    product_id_code = self._product_id_2_code(item_id)

    #implicit라이브러리의 explain 함수를 사용하여 결과값을 반환
    total_score, top_contributions, user_weights = self.model.to_cpu().explain(user_id_code, self.user_item_matrix, product_id_code)

    results = []
    categorys = []
    brands = []
    scores = []
    # id에 해당하는 user_id, product_id, column, category, brand를 찾기
    for id_, score_ in top_contributions:
      product_id = self._code_2_product_id(id_)
      result = self.data.loc[(self.data['product_id'] == product_id) & (self.data['user_id'] == user_id)][['user_id','product_id',column]]
      category = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'category_code'].unique()[0]
      brand = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'brand'].unique()[0]

      results.append(result)
      categorys.append(category)
      brands.append(brand)
      scores.append(score_)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame['score'] = scores
    frame['category'] = categorys
    frame['brand'] = brands
    
    frame = frame[['user_id', 'product_id','category','brand',column, 'score']]
    return frame, total_score

  def _get_train_test(self,percentage=.2, seed=42):
    """
    score를 구하기 위하여 train, test 데이터를 나누어주는 함수
    파라미터로 들어오는 percentage만큼 train_set의 값을 0으로 만들어 줌
    test_set는 기존의 User-Item Matrix에서 0이 아닌값으로 모두 1로 만들어 줌

    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율
    seed : random seed

    """
    #원본 데이터를 test_set, train_set에 복사
    test_set = self.user_item_matrix.copy()
    train_set = self.user_item_matrix.copy()

    #relevant(선호 혹은 평가)여부를 확인하기 위하여 test_set에서 0이 아닌값을 1로 만들어 줌 
    test_set[test_set != 0] = 1

    #train_set에서 0이 아닌 x축, y축을 추출
    nonzero_idxs = train_set.nonzero()
    #x, y를 짝을지어 저장
    nonzero_pairs = list(zip(nonzero_idxs[0], nonzero_idxs[1]))

    #랜덤 시드를 적용
    random.seed(seed)
    #주어진 비율로 샘플을 추출
    n_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, n_samples)

    
    user_idxs = [index[0] for index in samples]
    item_idxs = [index[1] for index in samples]

    #샘플에 해당 하는 값들을 평가한적이 없도록 보이기 위하여 0으로 감춤
    train_set[user_idxs, item_idxs] = 0
    
    train_set.eliminate_zeros()

    self.zero_user_idxs = user_idxs
    self.zero_item_idxs = item_idxs
    self.train_set = train_set
    self.test_set = test_set
    

  def get_score(self, percentage =.2, seed=42, k = 10, method='hit_at_k', n_samples = 10000):
    """
    train_set로 학습하고 test_set와 비교하여 method 파라미터를 이용하여 추천시스템의 성능을 평가하는 함수
    
    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율, default = .2
    seed : random seed, default = 42
    k : 추천할 아이템의 수, default = 10
    method : 평가 지표 - hit_at_k, precision_at_k
    n_samples : 평가할 user의 수, default = 10000

    """
    # 입력받은 n_samples가 최대값이상 이면 최대값으로 적용
    max_num = self.data['user_id_code'].nunique()
    if max_num <= n_samples:
      n_samples = max_num

    # train, test로 데이터가 분리
    self._get_train_test(percentage, seed)
    # metrics_model이 학습
    self.metrics_model.fit(self.train_set)
    # 샘플 user를 랜덤으로 추출
    random_state = np.random.RandomState(seed)
    user_id_code_samples = random_state.choice(self.data['user_id_code'], n_samples)

    #method 방법에 따라 scores 값을 반환
    if method == 'hit_at_k':
      scores = self._hit_at_k(user_id_code_samples, k)
    elif method =='presicion_at_k':
      scores = self._precision_at_k(user_id_code_samples, k)
    return scores

  def _hit_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 relevant한것(존재)이 있다면 1, 아니면 0을 반환하여 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]

      results = []
      # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 만약 결과 리스트안에 1이 있다면 1을 입력, 아니면 0을 입력
      if 1 in results:
        scores.append(1)
      else:
        scores.append(0)
        #결과를 평균내어 반환
    return np.mean(scores)

  def _precision_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 사용자가 relevant(선호 혹으 평가)한 아이템이 얼마나 존재하는지 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]
      results = []
       # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 유저가 추천받은 아이템들을 얼마나 선호(혹은 평가)했는지 추출
      scores.append(np.mean(results))
      # 결과를 평균내어 반환
    return np.mean(scores)

In [17]:
model = MyALS(AlternatingLeastSquares(), user_item_matrix, product_lookup, data)

In [18]:
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

# 추천하기

In [19]:
data.sort_values('view_count', ascending=False)[:10]

,user_id,product_id,view_count,user_id_code,product_id_code
243677,512475445,4700419,228,33856,15474
243682,512475445,4700478,225,33856,15503
243684,512475445,4700557,211,33856,15538
243687,512475445,4700590,206,33856,15559
243752,512475445,5800802,197,33856,21002
243703,512475445,5700788,193,33856,20353
5080310,519607186,1004887,192,559007,938
243725,512475445,5701086,184,33856,20473
243718,512475445,5701062,184,33856,20457
11519758,554252708,25600088,129,1561631,105511


아래의 데이터프레임은 view_counts가 높은 순으로 정렬한 데이터입니다.  
user_id가 512475445, 519607186가 높은 랭크를 기록하고 있습니다.

가장 많은 view_counts를 기록한 512475445 유저에게 추천하는 상품을 확인하겠습니다.

## 512475445 유저 확인

In [20]:
user = 512475445
column = 'view_count'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_count
868,4700419,auto.accessories.videoregister,sho-me,228
763,4700478,auto.accessories.videoregister,sho-me,225
4726,4700557,auto.accessories.videoregister,sho-me,211
4864,4700590,auto.accessories.videoregister,sho-me,206
14595,5800802,electronics.audio.subwoofer,kenwood,197
2019,5700788,auto.accessories.player,kenwood,193
2814,5701062,auto.accessories.player,pioneer,184
4495,5701086,auto.accessories.player,pioneer,184
487,5700384,auto.accessories.player,pioneer,127
23871,6100194,auto.accessories.radar,NaN,123


이 유저가 많이 본 제품은 videoregister, subwoofer, player 입니다.

이 유저에게 모델이 어떤 제품을 추천했는지 확인해보겠습니다.

In [21]:
model.get_recom_product(user)

,product_id,category_code,brand
365,1004794,electronics.smartphone,xiaomi
744,1004903,electronics.smartphone,huawei
1048,1003310,electronics.smartphone,apple
593,1004795,electronics.smartphone,xiaomi
252,1004246,electronics.smartphone,apple
4,1004237,electronics.smartphone,apple
334,4804056,electronics.audio.headphone,apple
96,1003549,electronics.smartphone,samsung
2267,5801218,electronics.audio.subwoofer,NaN
1236,1801766,electronics.video.tv,artel


해당 유저(512475445)에게는 smartphone과 subwoofer, headphone을 추천합니다.

맨 위에 위치한 1005104 id를 가진 제품을 어떤 이유로 추천했는지 확인해보겠습니다.

In [22]:
explain_frame, total_score = model.get_explain(user, 1005104, column)
explain_frame

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


,user_id,product_id,category,brand,view_count,score
243722,512475445,5701079,auto.accessories.player,jvc,100,0.097821
243687,512475445,4700590,auto.accessories.videoregister,sho-me,206,0.096863
243684,512475445,4700557,auto.accessories.videoregister,sho-me,211,0.093409
243780,512475445,6100194,auto.accessories.radar,NaN,123,0.076630
243700,512475445,5700619,auto.accessories.player,jvc,28,0.057814
243704,512475445,5700791,auto.accessories.player,sony,123,0.057243
243784,512475445,6100261,auto.accessories.radar,sho-me,94,0.046802
243779,512475445,6100193,auto.accessories.radar,sho-me,100,0.040805
243688,512475445,4700600,auto.accessories.videoregister,NaN,90,0.040682
243770,512475445,6000032,auto.accessories.alarm,cenmax,57,0.040177


## 519607186 유저 확인

In [23]:
user = 519607186
column = 'view_count'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_count
5969,1004887,electronics.smartphone,oppo,192
1166,1005159,electronics.smartphone,xiaomi,3
50,1002544,electronics.smartphone,apple,3
171,15100367,NaN,NaN,3
4187,14701391,furniture.living_room.cabinet,NaN,3
1259,14701558,furniture.living_room.cabinet,brw,3
19848,14701705,furniture.living_room.cabinet,brw,2
40247,14700394,furniture.living_room.cabinet,NaN,2
52673,14700738,furniture.living_room.cabinet,brw,2
2286,15100147,NaN,lider,2


이 유저는 192번이나 oppo의 스마트폰 제품을 보았습니다.

In [24]:
model.get_recom_product(user)

,product_id,category_code,brand
141,1004838,electronics.smartphone,oppo
585,1004839,electronics.smartphone,oppo
1113,1004886,electronics.smartphone,oppo
1055,1004961,electronics.smartphone,oppo
2392,1004990,electronics.smartphone,oppo
120205,1005205,electronics.smartphone,NaN
73,15100337,NaN,NaN
97,1004720,electronics.smartphone,huawei
524,1005021,electronics.smartphone,oppo
744,1004903,electronics.smartphone,huawei


알맞게 추천을 한것을 확인할 수 있습니다.

어떤 이유로 1004838 id값을 가진 제품을 추천했는지 확인해보겠습니다.

In [25]:
explain_frame, total_score = model.get_explain(user, 1004838, column)
explain_frame

,user_id,product_id,category,brand,view_count,score
5080310,519607186,1004887,electronics.smartphone,oppo,192,1.774710
5080352,519607186,15100367,NaN,NaN,3,0.033534
5080301,519607186,1002099,electronics.smartphone,samsung,1,0.017085
5080312,519607186,1005015,electronics.smartphone,samsung,1,0.012718
5080342,519607186,14701435,furniture.living_room.cabinet,NaN,1,0.011565
5080348,519607186,15100110,NaN,NaN,1,0.006555
5080353,519607186,17200570,furniture.living_room.sofa,NaN,1,0.006154
5080347,519607186,15100072,NaN,imperial,1,0.003896
5080345,519607186,14701705,furniture.living_room.cabinet,brw,2,0.003524
5080329,519607186,14700106,furniture.living_room.cabinet,NaN,1,0.002509


# 성능 평가하기

In [26]:
scores = model.get_score(method='hit_at_k')
print()
print('hit@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:12<00:00, 805.05it/s]


hit@k(10) :  0.27


In [27]:
scores = model.get_score(method='presicion_at_k')
print()
print('precision@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:12<00:00, 782.70it/s]


precision@k(10) :  0.04
